In [2]:
import re
import sys
from pathlib import Path
from dataclasses import dataclass
from typing import Optional
import logging
from collections import defaultdict
# logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)
logger = logging.getLogger(__name__)

# set current directory to Base directory
BASE_DIR = Path.cwd().parent
print("--------------------------------")
print(BASE_DIR)
print("--------------------------------")
# ============================================================================
# CONFIGURATION
# ============================================================================

FLIPPER_SIGNALS_DIR = BASE_DIR / "data/subghz"
OUTPUT_HEADER = BASE_DIR / "generated_signals.h"
OUTPUT_SOURCE = BASE_DIR / "generated_signals.cpp"

testfile = FLIPPER_SIGNALS_DIR / "TouchTunesPin/0.sub"
if not testfile.exists():
    print(f"File {testfile} does not exist")
    exit(1)
# ============================================================================
# DATA STRUCTURES
# ============================================================================

@dataclass
class FlipperSignal:
    """Represents a parsed Flipper .sub file"""

    name: str
    filename: str
    category: str
    frequency: float  # MHz
    raw_data: list[int]
    description: str = ""
    protocol: str = "RAW"
    preset: str = ""



--------------------------------
/home/cody/Code/Esp32SubGhz
--------------------------------


In [3]:
def parse_flipper_sub_file(
    filepath: Path, category: str = 'MISC'
) -> Optional[FlipperSignal]:
    """
    Parse a Flipper Zero .sub file and extract signal data.

    Flipper .sub file format:
        Filetype: Flipper SubGhz RAW File
        Version: 1
        Frequency: 433920000
        Preset: FuriHalSubGhzPresetOok650Async
        Protocol: RAW
        RAW_Data: 500 -1000 500 -500 ...
    """
    try:
        content = filepath.read_text(encoding="utf-8")
    except FileNotFoundError:
        logger.error("Could not read %s: File not found", filepath)
        return None
    except (OSError, UnicodeDecodeError) as e:
        logger.error("Could not read %s: %s", filepath, e)
        return None

    # Extract fields using regex
    frequency_match = re.search(r"Frequency:\s*(\d+)", content)
    protocol_match = re.search(r"Protocol:\s*(\w+)", content)
    preset_match = re.search(r"Preset:\s*(\S+)", content)

    # RAW_Data can span multiple generated_source_code
    raw_data_matches = re.findall(r"RAW_Data:\s*([-\d\s]+)", content)

    if not frequency_match:
        logger.warning("No frequency found in %s, skipping", filepath.name)
        return None

    if not raw_data_matches:
        logger.warning("No RAW_Data found in %s, skipping", filepath.name)
        return None

    # Parse frequency (Flipper uses Hz, we want MHz)
    frequency_hz = int(frequency_match.group(1))
    frequency_mhz = frequency_hz / 1_000_000.0

    # Parse all RAW_Data generated_source_code and combine
    raw_data = []
    for raw_line in raw_data_matches:
        values = raw_line.strip().split()
        for val in values:
            try:
                raw_data.append(int(val))
            except ValueError:
                continue

    if not raw_data:
        logger.warning("Empty RAW_Data in %s, skipping", filepath.name)
        return None

    # Generate name from filename
    name = filepath.stem.replace("_", " ").replace("-", " ").title()

    # Try to extract description from file comments or use filename
    desc_match = re.search(r"#\s*Description:\s*(.+)", content, re.IGNORECASE)
    description = (
        desc_match.group(1).strip()
        if desc_match
        else f"Signal from {filepath.name}"
    )

    return FlipperSignal(
        name=name,
        filename=filepath.stem,
        category=category,
        frequency=frequency_mhz,
        raw_data=raw_data,
        description=description,
        protocol=protocol_match.group(1) if protocol_match else "RAW",
        preset=preset_match.group(1) if preset_match else "",
    )

parse_flipper_sub_file(testfile)


FlipperSignal(name='0', filename='0', category='MISC', frequency=433.92, raw_data=[9056, -4528, 566, -566, 566, -1698, 566, -566, 566, -1698, 566, -1698, 566, -1698, 566, -566, 566, -1698, 566, -566, 566, -566, 566, -566, 566, -566, 566, -566, 566, -566, 566, -566, 566, -566, 566, -1698, 566, -566, 566, -566, 566, -1698, 566, -1698, 566, -566, 566, -566, 566, -566, 566, -566, 566, -1698, 566, -1698, 566, -566, 566, -566, 566, -1698, 566, -1698, 566, -1698, 566], description='Signal from 0.sub', protocol='RAW', preset='FuriHalSubGhzPresetOok650Async')

In [4]:

# ============================================================================
# DIRECTORY SCANNER
# ============================================================================
def scan_flipper_directory(flipper_signals_dir: Path) -> dict[str, list[FlipperSignal]]:
    """
    Scan a directory for .sub files organized into category folders.

    Returns a dictionary mapping category names to lists of FlipperSignal objects.
    """
    categories: dict[str, list[FlipperSignal]] = defaultdict(list)

    # Appended category directories within the specified path.
    category_dirs = []
    for category_dir in flipper_signals_dir.iterdir():
        if category_dir.is_dir() and not category_dir.name.startswith("."):
            category_dirs.append(category_dir)
    logger.info("Category_Dirs: %s", category_dirs)
    
    # scan subdirectories (categories) for .sub files
    for sub_file in category_dir.iterdir():
        if sub_file.name.endswith(".sub"):
            logger.debug("  → Parsing: %s", sub_file.name)
            signal = parse_flipper_sub_file(sub_file, category_dir.name)
            if signal:
                logger.info("Appending To Category Dict: %s", category_dir.name)
                categories[category_dir.name].append(signal)
                    
    for category_name, signals in categories.items():
        logger.info("Category: %s, Signals: %d", category_name, len(signals))
        for signal in signals:
            logger.info("  - Signal: %s, Frequency: %.6f MHz, Data Points: %d, Sample: %s", signal.name, signal.frequency, len(signal.raw_data), signal.raw_data[:10])

    return categories
test = scan_flipper_directory(FLIPPER_SIGNALS_DIR)
        

INFO: Category_Dirs: [PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Tesla'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Walgreens'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/TouchTunesBrute'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/CVS'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/TouchTunesPin'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Lowes')]
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Category: Lowes, Signals: 7
INFO:   - Signal: Lowes Wirecutting, Frequency: 303.875000 MHz, Data Points: 1733, Sample: [363, -103368, 99, -66, 67, -198, 131, -64, 129, -230]
INFO:   - Signal: Electrical, Frequency: 303.875000 MHz, Data Points: 1657, Sample: [595, -134, 165, -100, 99, -1520, 527, -100, 4258, -39

In [5]:
# ============================================================================
# C++ CODE GENERATOR
# ============================================================================


def sanitize_identifier(name: str) -> tuple:
    """Convert a name to a valid C++ identifier"""
    # Replace spaces and special chars with underscores
    identifier = re.sub(r"[^a-zA-Z0-9]", "_", name)
    # Remove consecutive underscores
    identifier = re.sub(r"_+", "_", identifier)
    # Remove leading/trailing underscores
    identifier = identifier.strip("_")
    # Ensure it doesn't start with a number
    if identifier and identifier[0].isdigit():
        identifier = "sig_" + identifier
    return identifier.lower()


generated_header_code = [
    "// Generated by flipper_to_cpp.py from Flipper Zero .sub files",
    "",
    "#ifndef GENERATED_SIGNALS_H",
    "#define GENERATED_SIGNALS_H",
    "",
    "#include <Arduino.h>",
    "#include <pgmspace.h>",
    '#include "signals.h"  // For SubGHzSignal and SubghzSignalList structs',
    "",
    "// ==================== ARRAY LENGTH CONSTANTS ",
    "",
]

generated_source_code = [
    "// ============================================================================",
    "// AUTO-GENERATED FILE - DO NOT EDIT MANUALLY",
    "// Generated by flipper_to_cpp.py from Flipper Zero .sub files",
    "",
    '#include "generated_signals.h"',
    "",
    "// ==================== SAMPLE DATA ARRAYS (STORED IN FLASH) ====================",
    "// PROGMEM stores these in flash memory instead of RAM",
    "// ESP32 reads PROGMEM automatically - no special read functions needed",
    "",
]

def generate_code(categories: dict[str, list[FlipperSignal]]) -> str:
    """Generate the C++ header file content"""
#
#  ------------------------------------------------------------------------
#   header File Generation
#  ------------------------------------------------------------------------
#
    for category_name, signals in categories.items():
        for signal in signals:
            # sample count
            signal_array_len = (
                f"LENGTH_{sanitize_identifier(category_name)}_"
                f"{sanitize_identifier(signal.name)}"
            ).upper()
            generated_header_code.append(
                f"constexpr uint16_t {signal_array_len} = {len(signal.raw_data)};"
                f"// Array Sample"
            )
            # sample
            signal_array = (
                f"samples_{sanitize_identifier(category_name)}_"
                f"{sanitize_identifier(signal.name)}"
            )
            generated_header_code.append(f"extern const int16_t {signal_array}[{len(signal.raw_data)}] PROGMEM;")
            signal_array = (
                f"samples_{sanitize_identifier(category_name)}_"
                f"{sanitize_identifier(signal.name)}"
            )
            
                        # Format raw data nicely (max ~8 values per line)
            data = []
            for i in range(0, len(signal.raw_data), 8):
                chunk = signal.raw_data[i : i + 8]
                data.append("    " + ", ".join(str(v) for v in chunk))
                
            generated_source_code.append(f"extern const int16_t {signal_array}[{len(signal.raw_data)}] PROGMEM = {{")
            generated_source_code.append(",\n".join(data))
            generated_source_code.append("};")
            generated_source_code.append("")

    # Generate category arrays of signals and their length constants
    generated_header_code.extend(["", "//================ Signal Category arrays"])
    # Generate sample arrays
    for category_name, signals in categories.items():
        
        category_array_len_variable = f"NUM_{sanitize_identifier(category_name).upper()}"
        generated_header_code.append(f"extern const uint8_t {category_array_len_variable};")
        
        category_array_name = f"SIGNALS_{sanitize_identifier(category_name).upper()}"
        generated_header_code.append(
            f"extern SubGHzSignal {category_array_name}[{category_array_len_variable}];"
            )
                

    generated_header_code.extend(
        [
            "",
            "// Generated Categories List",
            "extern SubghzSignalList GEN_SIGNAL_CATEGORIES[];",
            "extern const uint8_t NUM_GEN_CATEGORIES;",
            "",
            "#endif  // GENERATED_SIGNALS_H",
            "",
        ]
    )

#
#  ------------------------------------------------------------------------> tuple:
#   Source File Generation
#  ------------------------------------------------------------------------
#


    generated_source_code.extend(
        [
            "// ==================== SIGNAL Category STRUCTURES ====================",
            "",
        ]
    )

    # Generate signal arrays per category
    for category_name, signals in categories.items():
        array_name = f"SIGNALS_{sanitize_identifier(category_name).upper()}"
        generated_source_code.append(f"SubGHzSignal {array_name}[{len(signals)}] = {{")

        for i, signal in enumerate(signals):
            var_name = (
                f"samples_{sanitize_identifier(category_name)}_"
                f"{sanitize_identifier(signal.name)}"
            )
            const_name = (
                f"LENGTH_{sanitize_identifier(category_name)}_"
                f"{sanitize_identifier(signal.name)}"
            )

            # Escape quotes in name/description
            safe_name = signal.name.replace('"', '\\"')
            safe_desc = signal.description.replace('"', '\\"')

            comma = "," if i < len(signals) - 1 else ""
            generated_source_code.append(
                f'    {{"{safe_name}", "{safe_desc}", {var_name}, '
                f'{const_name}, {signal.frequency:.2f}f}}{comma}'
            )

        generated_source_code.append("};")

    generated_source_code.extend(
        [
            "// ==================== GENERATED CATEGORIES ====================",
            "",
            "SubghzSignalList SIGNAL_CATEGORIES[] = {",
        ]
    )

    # Generate categories list
    for i, (category_name, signals) in enumerate(categories.items()):
        array_name = f"SIGNALS_{sanitize_identifier(category_name).upper()}"
        count_name = f"NUM_{sanitize_identifier(category_name).upper()}"
        safe_name = category_name.replace('"', '\\"')
        comma = "," if i < len(categories) - 1 else ""
        generated_source_code.append(
            f'    {{"{safe_name}", {array_name}, {count_name}}}{comma}'
        )

    generated_source_code.extend(
        [
            "};",
            "",
            "const uint8_t NUM_OF_CATEGORIES = "
            "sizeof(SIGNAL_CATEGORIES) / sizeof(SubghzSignalList);",
            "",
        ]
    )
    
    header_code =  "\n".join(generated_header_code)
    source_code =  "\n".join(generated_source_code)
    return source_code, header_code


In [7]:
def main():
    # Scan for .sub files
    categories = scan_flipper_directory(FLIPPER_SIGNALS_DIR)

    if not categories:
        logger.warning("No categories found. Nothing to generate.")
        return 1

    # Generate C++ code
    generated_source_output, generated_header_output = generate_code(categories)

    # Write output files
    header_path = BASE_DIR / OUTPUT_HEADER
    source_path = BASE_DIR / OUTPUT_SOURCE

    try:
        header_path.write_text(generated_header_output, encoding="utf-8")
        logger.info("Generated: %s", header_path)
    except Exception as e:
        logger.error("Failed to write header file %s: %s", header_path, e)
        return 

    try:
        source_path.write_text(generated_source_output, encoding="utf-8")
        logger.info("Generated: %s", source_path)
    except Exception as e:
        logger.error("Failed to write source file %s: %s", source_path, e)
        return 

    logger.info("C++ code generation complete.")
    return 


if __name__ == "__main__":
    main()

INFO: Category_Dirs: [PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Tesla'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Walgreens'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/TouchTunesBrute'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/CVS'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/TouchTunesPin'), PosixPath('/home/cody/Code/Esp32SubGhz/data/subghz/Lowes')]
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Appending To Category Dict: Lowes
INFO: Category: Lowes, Signals: 7
INFO:   - Signal: Lowes Wirecutting, Frequency: 303.875000 MHz, Data Points: 1733, Sample: [363, -103368, 99, -66, 67, -198, 131, -64, 129, -230]
INFO:   - Signal: Electrical, Frequency: 303.875000 MHz, Data Points: 1657, Sample: [595, -134, 165, -100, 99, -1520, 527, -100, 4258, -39